# Frequency analysis module

In [1]:
# Basic imports
import xarray as xr
import hvplot.xarray
import numpy as np
import xdatasets as xd

import xhydro as xh
import xhydro.frequency_analysis as xhfa

c:\Users\PM217776\miniconda3\envs\xhydro-dev\Lib\site-packages\clisops\core\regrid.py:42: UserWarning: xarray version >= 2023.03.0 is not supported for regridding operations with cf-time indexed arrays. Please use xarray version < 2023.03.0. For more information, see: https://github.com/pydata/xarray/issues/7794.
  warnings.warn(
c:\Users\PM217776\miniconda3\envs\xhydro-dev\Lib\site-packages\xdatasets\spatial.py:14: UserWarning: xagg is not installed. Please install it with `pip install xagg`
  warnings.warn("xagg is not installed. Please install it with `pip install xagg`")


## Extracting and preparing the data
For this example, we'll conduct a frequency analysis using historical time series from various sites. We begin by obtaining a dataset comprising hydrological information. Here, we use the [xdataset](https://hydrologie.github.io/xdatasets/notebooks/getting_started.html) library to acquire hydrological data from the [Ministère de l'Environnement, de la Lutte contre les changements climatiques, de la Faune et des Parcs](https://www.cehq.gouv.qc.ca/atlas-hydroclimatique/stations-hydrometriques/index.htm) in Quebec, Canada. Specifically, our query focuses on stations with IDs beginning with `020`, possessing a natural flow pattern and limited to streamflow data. 

Users may prefer to generate their own `xarray.DataArray` using their individual dataset. At a minimum, the `xarray.DataArray` used for frequency analysis has to follow these principles:

- The dataset needs a `time` dimension.
- If there is a spatial dimension, such as `id` in the example below, it needs an attribute `cf_role` with `timeseries_id` as its value.
- The variable will at the very least need a `units` attribute, although other attributes such as `long_name` and `cell_methods` are also expected by `xclim` (which is called at various points during the frequency analysis) and warnings will be generated if they are missing.

In [2]:
ds = xd.Query(
    **{
        "datasets":{
            "deh":{
                "id" :["0510*"],
                "variables":["streamflow"],
            }
        }, "time":{"start": "1970-01-01", 
                   "minimum_duration":(15*365, 'd')},

  }
).data.squeeze().load()

# This dataset lacks some of the aforementioned attributes, so we need to add them.
ds["id"].attrs["cf_role"] = "timeseries_id"
ds["streamflow"].attrs = {"long_name": "Streamflow", "units": "m3 s-1", "standard_name": "water_volume_transport_in_river_channel", "cell_methods": "time: mean"}

ds

<xarray.Dataset>
Dimensions:        (id: 6, time: 20454)
Coordinates: (12/15)
    drainage_area  (id) float32 1.107e+03 3.8 9.1 1.1 267.0 3.7
    end_date       (id) datetime64[ns] 2024-02-03 2024-02-01 ... 2024-02-06
  * id             (id) object '051001' '051002' '051003' ... '051005' '051007'
    latitude       (id) float32 46.9 47.27 47.27 47.29 47.26 47.27
    longitude      (id) float32 -71.15 -71.16 -71.14 -71.16 -71.14 -71.16
    name           (id) object 'Montmorency' ... 'Des Aulnaies'
    ...             ...
    spatial_agg    <U9 'watershed'
    start_date     (id) datetime64[ns] 1970-01-01 1970-01-01 ... 1971-11-15
  * time           (time) datetime64[ns] 1970-01-01 1970-01-02 ... 2025-12-31
    time_agg       <U4 'mean'
    timestep       <U1 'D'
    variable       <U10 'streamflow'
Data variables:
    streamflow     (id, time) float32 9.91 9.49 9.12 8.78 ... nan nan nan nan

In [3]:
ds.streamflow.dropna('time', 'all').hvplot(x='time', grid=True, widget_location='bottom', groupby='id')

C:\Users\PM217776\AppData\Local\Temp\ipykernel_18580\3990158135.py:1: FutureWarning: Passing 'how' as positional argument(s) to dropna was deprecated in version v2023.10.0 and will raise an error two releases later. Please pass them as keyword arguments.


BokehModel(combine_events=True, render_bundle={'docs_json': {'f18f70f3-1d38-4ce0-be97-fb81dcf78b01': {'version…

## Customizing the analysis settings

### a) Defining seasons
We can define seasons using indexers that are compatible with `xclim.core.calendar.select_time`. There are currently four accepted types of indexers:

- `month`, followed by a sequence of month numbers.
- `season`, followed by one or more of 'DJF', 'MAM', 'JJA' and 'SON'.
- `doy_bounds`, followed by a sequence representing the inclusive bounds of the period to be considered (start, end).
- `date_bounds`, which is the same as above, but using a month-day (%m-%d) format.

For the purpose of getting block maxima through `xhydro.indicators.get_yearly_op`, the indexers need to be grouped within a dictionary, with the key being the label to be given to the requested period of the year. A second key can be used to instruct on the resampling frequency, for example to wrap around the year for winter.

In [4]:
# Some examples
timeargs = {
    "spring": {"date_bounds": ["02-11", "06-19"]},
    "summer": {"doy_bounds": [152, 243]},
    "fall": {"month": [9, 10, 11]},
    "winter": {"season": ['DJF'], "freq": "AS-DEC"},  # To correctly wrap around the year, we need to specify the resampling frequency.
    "august": {"month": [8]},
    "annual": {}
    }

### b) Getting block maxima
Upon selecting each desired season, we can extract block maxima timeseries from every station using `xhydro.indicators.get_yearly_op`. The main arguments are:

- `op`: the operation to compute. One of "max", "min", "mean", "sum".
- `input_var`: the name of the variable. Defaults to "streamflow".
- `window`: the size of the rolling window. A "mean" is performed on the rolling window prior to the `op` operation.
- `timeargs`: as defined previously. Leave at `None` to get the annual maxima.
- `missing` and `missing_options`: to define tolerances for missing data. See [this page](https://xclim.readthedocs.io/en/stable/checks.html#missing-values-identification) for more information.
- `interpolate_na`: whether to interpolate missing data prior to the `op` operation. Only used for `sum`.

The function returns a `xarray.Dataset` with 1 variable per indexer.

In [5]:
# Here, we hide years with more than 15% of missing data.
ds_4fa = xh.indicators.get_yearly_op(ds, op="max", timeargs=timeargs, missing="pct", missing_options={"tolerance": 0.15})

ds_4fa

c:\Users\PM217776\miniconda3\envs\xhydro-dev\Lib\site-packages\xclim\core\calendar.py:801: FutureWarning: 'AS-JAN' is deprecated and will be removed in a future version, please use 'YS-JAN' instead.
c:\Users\PM217776\miniconda3\envs\xhydro-dev\Lib\site-packages\xclim\core\calendar.py:801: FutureWarning: 'AS-DEC' is deprecated and will be removed in a future version, please use 'YS-DEC' instead.
c:\Users\PM217776\miniconda3\envs\xhydro-dev\Lib\site-packages\xarray\core\groupby.py:534: FutureWarning: 'AS-JAN' is deprecated and will be removed in a future version, please use 'YS-JAN' instead.
c:\Users\PM217776\miniconda3\envs\xhydro-dev\Lib\site-packages\xarray\core\groupby.py:534: FutureWarning: 'AS-JAN' is deprecated and will be removed in a future version, please use 'YS-JAN' instead.
c:\Users\PM217776\miniconda3\envs\xhydro-dev\Lib\site-packages\xclim\core\missing.py:141: FutureWarning: 'AS-JAN' is deprecated and will be removed in a future version, please use 'YS-JAN' instead.
c:\Use

<xarray.Dataset>
Dimensions:                (id: 6, time: 56)
Coordinates: (12/15)
    drainage_area          (id) float32 1.107e+03 3.8 9.1 1.1 267.0 3.7
    end_date               (id) datetime64[ns] 2024-02-03 ... 2024-02-06
  * id                     (id) object '051001' '051002' ... '051005' '051007'
    latitude               (id) float32 46.9 47.27 47.27 47.29 47.26 47.27
    longitude              (id) float32 -71.15 -71.16 -71.14 ... -71.14 -71.16
    name                   (id) object 'Montmorency' ... 'Des Aulnaies'
    ...                     ...
    spatial_agg            <U9 'watershed'
    start_date             (id) datetime64[ns] 1970-01-01 ... 1971-11-15
    time_agg               <U4 'mean'
    timestep               <U1 'D'
    variable               <U10 'streamflow'
  * time                   (time) datetime64[ns] 1970-01-01 ... 2025-01-01
Data variables:
    streamflow_max_spring  (id, time) float32 331.0 294.0 354.0 ... nan nan
    streamflow_max_summer  (id, time) float32 261.0 161.0 354.0 ... nan nan
    streamflow_max_fall    (id, time) float32 154.0 151.0 176.0 ... nan nan
    streamflow_max_august  (id, time) float32 76.5 161.0 183.0 ... 0.419 nan nan
    streamflow_max_annual  (id, time) float32 331.0 294.0 354.0 ... nan nan
    streamflow_max_winter  (id, time) float32 19.8 17.4 112.0 ... nan nan nan
Attributes:
    cat:frequency:         None
    cat:processing_level:  indicators
    cat:id:

In [6]:
ds_4fa.streamflow_max_summer.dropna('time', 'all').hvplot(x='time', grid=True, widget_location='bottom', groupby='id')

C:\Users\PM217776\AppData\Local\Temp\ipykernel_18580\3578841157.py:1: FutureWarning: Passing 'how' as positional argument(s) to dropna was deprecated in version v2023.10.0 and will raise an error two releases later. Please pass them as keyword arguments.


BokehModel(combine_events=True, render_bundle={'docs_json': {'2fe877fd-0403-4bf4-891c-455235e6ee24': {'version…

### c) Using custom seasons per year or per station

Using individualized date ranges for each year or each catchment is not explicitely supported, so users should instead mask their data prior to calling `get_yearly_op`. Note that when doing this, `missing` should be adjusted accordingly.

In [7]:
# Create a mask beforehand
import random
nyears = np.unique(ds.time.dt.year).size
dom_start = xr.DataArray(np.random.randint(1, 30, size=(nyears, )).astype("str"), dims=("year"), coords={"year": np.unique(ds.time.dt.year)})
dom_end = xr.DataArray(np.random.randint(1, 30, size=(nyears, )).astype("str"), dims=("year"), coords={"year": np.unique(ds.time.dt.year)})

mask = xr.zeros_like(ds["streamflow"])
for y in dom_start.year.values:
    # Random mask of dates per year, between April and June.
    mask.loc[{"time": slice(str(y)+"-04-"+str(dom_start.sel(year=y).item()), str(y)+"-06-"+str(dom_end.sel(year=y).item()))}] = 1

In [8]:
mask.hvplot(x='time', grid=True, widget_location='bottom', groupby='id')

BokehModel(combine_events=True, render_bundle={'docs_json': {'da744136-9397-4335-99a9-2be94b034345': {'version…

In [9]:
# The name of the indexer will be used to identify the variable created here
timeargs_custom = {"custom": {}}

# We use where() to mask the data that we want to ignore
masked = ds.where(mask==1)
# Since we masked almost all of the year, our tolerance for missing data should be changed accordingly
missing = "at_least_n"
missing_options = {"n": 45}

# We use xr.merge() to combine the results with the previous dataset.
ds_4fa = xr.merge([ds_4fa, xh.indicators.get_yearly_op(masked, op="max", timeargs=timeargs_custom, missing=missing, missing_options=missing_options)])

c:\Users\PM217776\miniconda3\envs\xhydro-dev\Lib\site-packages\xclim\core\calendar.py:801: FutureWarning: 'AS-JAN' is deprecated and will be removed in a future version, please use 'YS-JAN' instead.
c:\Users\PM217776\miniconda3\envs\xhydro-dev\Lib\site-packages\xarray\core\groupby.py:534: FutureWarning: 'AS-JAN' is deprecated and will be removed in a future version, please use 'YS-JAN' instead.
c:\Users\PM217776\miniconda3\envs\xhydro-dev\Lib\site-packages\xarray\core\groupby.py:534: FutureWarning: 'AS-JAN' is deprecated and will be removed in a future version, please use 'YS-JAN' instead.


In [10]:
ds_4fa.streamflow_max_custom.dropna('time', 'all').hvplot(x='time', grid=True, widget_location='bottom', groupby='id')

C:\Users\PM217776\AppData\Local\Temp\ipykernel_18580\460065927.py:1: FutureWarning: Passing 'how' as positional argument(s) to dropna was deprecated in version v2023.10.0 and will raise an error two releases later. Please pass them as keyword arguments.


BokehModel(combine_events=True, render_bundle={'docs_json': {'9a2fcaf8-5fa0-488e-80f2-f94d6d364f80': {'version…

### d) Computing volumes

The frequency analysis can also be performed on volumes, using a similar workflow. The main difference is that if we're starting from streamflow, we'll first need to convert them into volumes using `xhydro.indicators.compute_volume`. Also, if required, `get_yearly_op` has an argument `interpolate_na` that can be used to interpolate missing data prior to the sum.

In [11]:
# Get a daily volume from a daily streamflow
ds["volume"] = xh.indicators.compute_volume(ds["streamflow"], out_units="hm3")

# We'll take slightly different indexers
timeargs_vol = {
    "spring": {"date_bounds": ["04-30", "06-15"]},
    "annual": {}
    }

# The operation that we want here is the sum, not the max.
ds_4fa = xr.merge([ds_4fa, xh.indicators.get_yearly_op(ds, op="sum", input_var="volume", timeargs=timeargs_vol, missing="pct", missing_options={"tolerance": 0.15}, interpolate_na=True)])

c:\Users\PM217776\miniconda3\envs\xhydro-dev\Lib\site-packages\xclim\core\calendar.py:801: FutureWarning: 'AS-JAN' is deprecated and will be removed in a future version, please use 'YS-JAN' instead.
c:\Users\PM217776\miniconda3\envs\xhydro-dev\Lib\site-packages\xarray\core\groupby.py:534: FutureWarning: 'AS-JAN' is deprecated and will be removed in a future version, please use 'YS-JAN' instead.
c:\Users\PM217776\miniconda3\envs\xhydro-dev\Lib\site-packages\xclim\core\formatting.py:131: UserWarning: Requested formatting `noun` for unknown string `integral`.
c:\Users\PM217776\miniconda3\envs\xhydro-dev\Lib\site-packages\xarray\core\groupby.py:534: FutureWarning: 'AS-JAN' is deprecated and will be removed in a future version, please use 'YS-JAN' instead.
c:\Users\PM217776\miniconda3\envs\xhydro-dev\Lib\site-packages\xclim\core\missing.py:141: FutureWarning: 'AS-JAN' is deprecated and will be removed in a future version, please use 'YS-JAN' instead.
c:\Users\PM217776\miniconda3\envs\xhydro

In [12]:
ds_4fa.volume_sum_spring.dropna('time', 'all').hvplot(x='time', grid=True, widget_location='bottom', groupby='id')

C:\Users\PM217776\AppData\Local\Temp\ipykernel_18580\4256194885.py:1: FutureWarning: Passing 'how' as positional argument(s) to dropna was deprecated in version v2023.10.0 and will raise an error two releases later. Please pass them as keyword arguments.


BokehModel(combine_events=True, render_bundle={'docs_json': {'88e84cd9-0261-4164-b9df-f7a772ef51c8': {'version…

## Local frequency analysis

Once we have our yearly maximums (or volumes/minimums), the first step in a local frequency analysis is to call `xhfa.local.fit` to obtain distribution parameters. The options are:

- `distributions`: list of [SciPy distributions](https://docs.scipy.org/doc/scipy/reference/stats.html#continuous-distributions). Defaults to ["expon", "gamma", "genextreme", "genpareto", "gumbel_r", "pearson3", "weibull_min"].
- `min_years`: minimum number of years required to fit the data.
- `method`: fitting method. Defaults to the maximum likelihood.

In [13]:
# To speed up the Notebook, we'll only perform the analysis on a subset of variables
params = xhfa.local.fit(ds_4fa[["streamflow_max_spring", "volume_sum_spring"]], min_years=15)

params

<xarray.Dataset>
Dimensions:                (id: 6, dparams: 5, scipy_dist: 7)
Coordinates: (12/16)
  * id                     (id) object '051001' '051002' ... '051005' '051007'
  * dparams                (dparams) <U5 'a' 'c' 'skew' 'loc' 'scale'
  * scipy_dist             (scipy_dist) <U11 'expon' 'gamma' ... 'weibull_min'
    drainage_area          (id) float32 1.107e+03 3.8 9.1 1.1 267.0 3.7
    end_date               (id) datetime64[ns] 2024-02-03 ... 2024-02-06
    latitude               (id) float32 46.9 47.27 47.27 47.29 47.26 47.27
    ...                     ...
    source                 <U102 'Ministère de l’Environnement, de la Lutte c...
    spatial_agg            <U9 'watershed'
    start_date             (id) datetime64[ns] 1970-01-01 ... 1971-11-15
    time_agg               <U4 'mean'
    timestep               <U1 'D'
    variable               <U10 'streamflow'
Data variables:
    streamflow_max_spring  (scipy_dist, id, dparams) float64 dask.array<chunksize=(1, 6, 5), meta=np.ndarray>
    volume_sum_spring      (scipy_dist, id, dparams) float64 dask.array<chunksize=(1, 6, 5), meta=np.ndarray>
Attributes:
    cat:frequency:         None
    cat:processing_level:  indicators
    cat:id:

Information Criteria such as the AIC, BIC, and AICC are useful to determine which statistical distribution is better suited to a given location. These three criteria can be computed using `xhfa.local.criteria`.

In [14]:
criteria = xhfa.local.criteria(ds_4fa[["streamflow_max_spring", "volume_sum_spring"]], params)

criteria

<xarray.Dataset>
Dimensions:                (id: 6, scipy_dist: 7, criterion: 3)
Coordinates: (12/16)
    drainage_area          (id) float32 1.107e+03 3.8 9.1 1.1 267.0 3.7
    end_date               (id) datetime64[ns] 2024-02-03 ... 2024-02-06
  * id                     (id) object '051001' '051002' ... '051005' '051007'
    latitude               (id) float32 46.9 47.27 47.27 47.29 47.26 47.27
    longitude              (id) float32 -71.15 -71.16 -71.14 ... -71.14 -71.16
    name                   (id) object 'Montmorency' ... 'Des Aulnaies'
    ...                     ...
    start_date             (id) datetime64[ns] 1970-01-01 ... 1971-11-15
    time_agg               <U4 'mean'
    timestep               <U1 'D'
    variable               <U10 'streamflow'
  * scipy_dist             (scipy_dist) <U11 'expon' 'gamma' ... 'weibull_min'
  * criterion              (criterion) <U4 'aic' 'bic' 'aicc'
Data variables:
    volume_sum_spring      (scipy_dist, id, criterion) float64 dask.array<chunksize=(1, 6, 3), meta=np.ndarray>
    streamflow_max_spring  (scipy_dist, id, criterion) float64 dask.array<chunksize=(1, 6, 3), meta=np.ndarray>
Attributes:
    cat:frequency:         None
    cat:processing_level:  indicators
    cat:id:

Finally, return periods can be obtained using `xhfa.local.parametric_quantiles`. The options are:

- `t`: the return period(s) in years.
- `mode`: whether the return period is the probability of exceedance (max) or non-exceedance (min). Defaults to `max`.

In [15]:
rp = xhfa.local.parametric_quantiles(params, t=[20, 100])

rp.load()

<xarray.Dataset>
Dimensions:                (id: 6, scipy_dist: 7, return_period: 2)
Coordinates: (12/17)
  * id                     (id) object '051001' '051002' ... '051005' '051007'
  * scipy_dist             (scipy_dist) <U11 'expon' 'gamma' ... 'weibull_min'
    drainage_area          (id) float32 1.107e+03 3.8 9.1 1.1 267.0 3.7
    end_date               (id) datetime64[ns] 2024-02-03 ... 2024-02-06
    latitude               (id) float32 46.9 47.27 47.27 47.29 47.26 47.27
    longitude              (id) float32 -71.15 -71.16 -71.14 ... -71.14 -71.16
    ...                     ...
    start_date             (id) datetime64[ns] 1970-01-01 ... 1971-11-15
    time_agg               <U4 'mean'
    timestep               <U1 'D'
    variable               <U10 'streamflow'
  * return_period          (return_period) int32 20 100
    p_quantile             (return_period) float64 0.95 0.99
Data variables:
    streamflow_max_spring  (scipy_dist, return_period, id) float64 788.0 ... ...
    volume_sum_spring      (scipy_dist, return_period, id) float64 942.7 ... ...
Attributes:
    cat:frequency:         None
    cat:processing_level:  indicators
    cat:id:

`xhfa.local._prepare_plots` generates points for later plotting xmin and xmax represent return_period limits<br>
If log=True, returns log-spaced x values between xmin and xmax.
Otherwise returns linearly spaced values between xmin and xmax.<br> In a future release, this function will be called from a plotting function but for now, we're showing the example on how to use it in this Notebook


In [21]:
data = xhfa.local._prepare_plots(params, xmin=1, xmax = 1000, npoints=50, log=True)
data.load()

<xarray.Dataset>
Dimensions:                (id: 6, scipy_dist: 7, return_period: 50)
Coordinates: (12/17)
  * id                     (id) object '051001' '051002' ... '051005' '051007'
  * scipy_dist             (scipy_dist) <U11 'expon' 'gamma' ... 'weibull_min'
    drainage_area          (id) float32 1.107e+03 3.8 9.1 1.1 267.0 3.7
    end_date               (id) datetime64[ns] 2024-02-03 ... 2024-02-06
    latitude               (id) float32 46.9 47.27 47.27 47.29 47.26 47.27
    longitude              (id) float32 -71.15 -71.16 -71.14 ... -71.14 -71.16
    ...                     ...
    start_date             (id) datetime64[ns] 1970-01-01 ... 1971-11-15
    time_agg               <U4 'mean'
    timestep               <U1 'D'
    variable               <U10 'streamflow'
  * return_period          (return_period) float64 1.0 1.151 ... 868.5 1e+03
    p_quantile             (return_period) float64 0.0 0.1315 ... 0.9988 0.999
Data variables:
    streamflow_max_spring  (scipy_dist, return_period, id) float64 104.6 ... ...
    volume_sum_spring      (scipy_dist, return_period, id) float64 119.7 ... ...
Attributes:
    cat:frequency:         None
    cat:processing_level:  indicators
    cat:id:

`xhfa.local._get_plotting_positions` allows you to get ploting positions for all variables in the dataset.
It accpets alpha and beta in arguments, see scipy.stats.mstats.plotting_positions for typical values for alpha and beta.
<br>By default (.4,.4) : approximately quantile unbiased (Cunnane)<br> In a future release, this function will be called from a plotting function but for now, we're showing the example on how to use it in this Notebook

    

In [17]:
pp = xhfa.local._get_plotting_positions(ds_4fa[['streamflow_max_spring']])
pp

<xarray.Dataset>
Dimensions:                   (id: 6, time: 56)
Coordinates: (12/15)
    drainage_area             (id) float32 1.107e+03 3.8 9.1 1.1 267.0 3.7
    end_date                  (id) datetime64[ns] 2024-02-03 ... 2024-02-06
  * id                        (id) object '051001' '051002' ... '051007'
    latitude                  (id) float32 46.9 47.27 47.27 47.29 47.26 47.27
    longitude                 (id) float32 -71.15 -71.16 ... -71.14 -71.16
    name                      (id) object 'Montmorency' ... 'Des Aulnaies'
    ...                        ...
    spatial_agg               <U9 'watershed'
    start_date                (id) datetime64[ns] 1970-01-01 ... 1971-11-15
    time_agg                  <U4 'mean'
    timestep                  <U1 'D'
    variable                  <U10 'streamflow'
  * time                      (time) datetime64[ns] 1970-01-01 ... 2025-01-01
Data variables:
    streamflow_max_spring_pp  (id, time) float64 2.0 1.343 2.86 ... nan nan
    streamflow_max_spring     (id, time) float32 331.0 294.0 354.0 ... nan nan

In [22]:
#Lets plot the observations
p1 = data.streamflow_max_spring.hvplot(x='return_period', by='scipy_dist', grid=True, groupby=['id'], logx=True) 

In [19]:
#Lets now plot the distributions
p2 = pp.hvplot.scatter(x='streamflow_max_spring_pp',y='streamflow_max_spring', grid=True, groupby=['id'], logx=True)

In [23]:
#And now combining the plots
p1 * p2

BokehModel(combine_events=True, render_bundle={'docs_json': {'4e726008-c445-4bda-8df2-21e6ba67e1bb': {'version…